In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline 


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense , Dropout , Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator 
from keras.callbacks import ReduceLROnPlateau 


In [ ]:
# load data 
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test= pd.read_csv("/kaggle/input/digit-recognizer/test.csv")



In [ ]:
y_train = train["label"]
x_train = train.drop(["label"],axis = 1)
# free up space
del train
# let's look at the data
x_train

In [ ]:
y_train

In [ ]:
# visualize data distribution
# similar distribution amogst different digits, so we are good here 
sns.countplot(y_train)

In [ ]:
# check for null values
print("count of null values in x train set: ",x_train.isnull().sum().sum())
print("count of null values in y train set: ",y_train.isnull().sum().sum())
print("count of null values in test set: ",test.isnull().sum().sum())

In [ ]:
# normalize data
x_train = x_train / 255.0
test = test / 255.0
# reshape data
x_train = x_train.values.reshape(-1, 28,28,1)
test = test.values.reshape(-1,28,28,1)

In [ ]:
# look at some example

plt.imshow(x_train[0][:,:,0],  cmap=plt.get_cmap('gray'))

In [ ]:
y_train = to_categorical(y_train, num_classes = 10)
# splitting data into train and test 
x_train, x_test , y_train,y_test = train_test_split(x_train,y_train,test_size = 0.1,random_state = 2)

In [ ]:
# CNN model
#  [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same', activation = 'relu'))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))

model.add(MaxPool2D(pool_size = (2,2) , strides =(2,2) ))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
# define the optimizer
optimizer = RMSprop(lr = 0.001, rho = 0.9 , epsilon = 1e-08, decay = 0.0 )
# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
# set a learning rate annealer 
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 3, verbose = 1, factor = 0.5, min_lr = 0.0001)
epochs = 30
batch_size = 86

In [ ]:
# data augmentation to prevent over fitting 

aug = ImageDataGenerator(rotation_range = 10, zoom_range = 0.15, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.15 )
#aug.fit(x_train)


In [ ]:
fit_model = model.fit_generator(aug.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 2, steps_per_epoch=x_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

In [ ]:
fit_model.history

In [ ]:
# visualize the loss value and the accuracy of train and test set
fig, ax = plt.subplots(2,1)
ax[0].plot(fit_model.history['loss'],color = 'b',label = 'train loss' )
ax[0].plot(fit_model.history['val_loss'],color = 'r',label = 'test loss' , axes = ax[0])

ax[1].plot(fit_model.history['accuracy'],color = 'b',label = 'train acc' )
ax[1].plot(fit_model.history['val_accuracy'],color = 'r',label = 'test acc' , axes = ax[1])

In [ ]:
Y_pred = model.predict(test)

In [ ]:
Y_pred
result = np.argmax(Y_pred,axis =1)

In [ ]:
result

In [ ]:
submission_df = pd.DataFrame(data = result, columns = ['Label'])

In [ ]:
submission_df['ImageId'] = pd.Series(range(1,28001))

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv("cnn_mnist_datagen.csv",index=False)